# This is almost the same as svm.ipynb, but writes out the write_submission_file function so you see output along the way.  Please double-check

In [9]:
import pandas as pd
import numpy as np
    
tourney_df = pd.read_csv("data/data.csv", index_col=0)
tourney_df.head()

,Season,Daynum,Wteam,Wscore,Lteam,Lscore,Wloc,Numot,Wfgm,Wfga,...,std_stl0,std_stl1,avg_blk0,avg_blk1,std_blk0,std_blk1,avg_pf0,avg_pf1,std_pf0,std_pf1
0,2003,134,1421,92,1411,84,N,1,32,69,...,2.31,3.14,2.23,3.00,1.72,1.65,18.30,19.10,4.56,3.69
1,2003,136,1112,80,1436,51,N,0,31,66,...,3.65,3.50,4.21,2.97,2.01,1.72,17.75,15.90,2.99,4.05
2,2003,136,1113,84,1272,71,N,0,31,59,...,2.14,3.16,4.24,5.07,2.94,3.16,19.41,18.76,3.25,4.34
3,2003,136,1141,79,1166,73,N,0,29,53,...,2.82,2.94,4.45,4.00,2.22,2.35,17.27,20.97,3.12,4.81
4,2003,136,1143,76,1301,74,N,1,27,64,...,2.64,3.63,2.79,3.07,1.63,2.49,17.10,18.67,3.74,4.21


In [10]:
featureList = ['wins0', 'wins1']
y = tourney_df['team0Win'].values # results
X = tourney_df[featureList].values # features

In [6]:
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=1)

In [7]:
from sklearn import svm
model = svm.SVC(probability=True)
model.fit(X_train, y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [8]:
y_pred = model.predict_proba(X_test)[:,0] # probability that team0 wins (what Kaggle calls team 1, and wants for submission)
for i in range(5):
    print("Features = {0}, Pred. prob. team0 wins: {1}".format(X_test[i], y_pred[i]))

Features = [26 30], Pred. prob. team0 wins: 0.5
Features = [33 25], Pred. prob. team0 wins: 0.427897404805
Features = [27 32], Pred. prob. team0 wins: 0.511082010237
Features = [22 24], Pred. prob. team0 wins: 0.378781317676
Features = [29 26], Pred. prob. team0 wins: 0.5


# Could you double-check I'm calculating this correctly?

Just realized Kaggle uses the convention for scoring that the label is 1 if the first team wins, and 0 if the second.  We use 0 if team0 wins, 1 if team1 wins.  I think what's below works...passing y_pred, i.e. the probability that the first team wins (i.e. what we call team0, what they call team 1), and 1-y_test, so that 1-y = 0 when team1 wins (their team2) and 1-y = 1 when team0 wins (their team1).  Please compare http://scikit-learn.org/stable/modules/generated/sklearn.metrics.log_loss.html and https://www.kaggle.com/c/march-machine-learning-mania-2016/details/evaluation to make sure we're doing the same thing.  It's late and I'm not thinking straight.

In [6]:
from sklearn import metrics
print("Log loss is {0}".format(metrics.log_loss(1-y_test, y_pred)))

Log loss is 0.6445127649406421


In [7]:
import pandas as pd
sample = pd.read_csv('data/sample.csv', index_col=0)

In [8]:
sample.head()

,Id,Pred,Season,team0,team1,wins0,wins1
0,2012_1104_1124,0.5,2012,1104,1124,21,26
1,2012_1104_1125,0.5,2012,1104,1125,21,26
2,2012_1104_1140,0.5,2012,1104,1140,21,23
3,2012_1104_1143,0.5,2012,1104,1143,21,24
4,2012_1104_1153,0.5,2012,1104,1153,21,24


In [9]:
Xsample = sample[featureList].values

In [10]:
sample['Pred'] = model.predict_proba(Xsample)[:,0]
sample.head(10)

,Id,Pred,Season,team0,team1,wins0,wins1
0,2012_1104_1124,0.379859,2012,1104,1124,21,26
1,2012_1104_1125,0.379859,2012,1104,1125,21,26
2,2012_1104_1140,0.385549,2012,1104,1140,21,23
3,2012_1104_1143,0.377617,2012,1104,1143,21,24
4,2012_1104_1153,0.377617,2012,1104,1153,21,24
5,2012_1104_1160,0.555934,2012,1104,1160,21,21
6,2012_1104_1161,0.607566,2012,1104,1161,21,20
7,2012_1104_1163,0.607566,2012,1104,1163,21,20
8,2012_1104_1166,0.354267,2012,1104,1166,21,28
9,2012_1104_1172,0.377617,2012,1104,1172,21,24


In [11]:
submission = sample[['Id', 'Pred']]
submission.head()

,Id,Pred
0,2012_1104_1124,0.379859
1,2012_1104_1125,0.379859
2,2012_1104_1140,0.385549
3,2012_1104_1143,0.377617
4,2012_1104_1153,0.377617


In [12]:
submission.to_csv("data/submission.csv", encoding='ascii')